# #148  Торговля по выходным, по утрам и вечерам

In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
from app import create_app
from app.models import Instrument
from app.config import RunConfig
from bot.env.prod import TimeProdEnvHelper, LoggerHelper, TinkoffProxyClient, AccountingHelper
from bot import TradingBot

app = create_app()
with app.app_context():
    TICKER = 'RNFT'
    INSTRUMENT_ID = 4
    
    log_name = TICKER
    
    instrument = Instrument.get_by_id(INSTRUMENT_ID)
    account_id = str(instrument.account)
    
    
    time_helper = TimeProdEnvHelper()
    logger_helper = LoggerHelper(__name__, log_name)
    client_helper = TinkoffProxyClient(TICKER, time_helper, logger_helper, account_id)
    accounting_helper = AccountingHelper('test', client_helper, time_helper)
    
    bot = TradingBot(
        config=RunConfig(
            ticker=TICKER,
    
            step_max_cnt=3,
            step_base_cnt=0,
            step_set_orders_cnt=2,
            step_size=1.2,
            step_lots=1,
    
            pretest_period=0,
            pretest_type=RunConfig.PRETEST_NONE,
    
            threshold_sell_steps=0,
            threshold_buy_steps=6,
    
            majority_trade=True,
    
            stop_up_p=.05,
            stop_down_p=.15,
    
            instrument_id=INSTRUMENT_ID,
        ),
        time_helper=time_helper,
        logger_helper=logger_helper,
        client_helper=client_helper,
        accounting_helper=accounting_helper,
    )

2024-11-17 00:35:53,573 - INFO - INIT 
     config - RNFT+ 3/0/2 x l1 x 1.2¤ |s0 b6| |u0.05 d0.15| 
     instrument - RNFT / BBG00F9XX7H4, РуссНефть, rub, round_signs=2, inc=0.05, lot=1, short=True
     cur_used_cnt - 4
     last_price - 137.6
     depo - 825.6
     instrument_id - 4
     run_instance - <Run 115 i4 (RNFT+ 3/0/2 x l1 x 1.2¤ |s0 b6| |u0.05 d0.15| ) at 2024-11-16>


In [13]:
with app.app_context():
    bot.update_run_state()
    print(bot.client.can_trade())
    print(bot.client.status)

True
GetTradingStatusResponse(figi='BBG00F9XX7H4', trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING: 1>, limit_order_available_flag=False, market_order_available_flag=False, api_trade_available_flag=True, instrument_uid='c7485564-ed92-45fd-a724-1214aa202904', bestprice_order_available_flag=False, only_best_price=False)


In [15]:
buy_res = bot.trade_strategy.buy(1)
print(buy_res)

2024-11-17 15:29:02,726 - ERROR - Не выставлена заявка: order_type: market, direction: buy, lots: 1, price: None, Error: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30068', Metadata(tracking_id='f9f91780a01c111748f99d362c6a0527', ratelimit_limit='300, 300;w=60', ratelimit_remaining=299, ratelimit_reset=58, message='Only limit order is allowed'))
2024-11-17 15:29:02,730 - ERROR - RETRY order. 1, 1, None, 2, sleep 5, retry num=1
2024-11-17 15:29:08,560 - ERROR - Не выставлена заявка: order_type: market, direction: buy, lots: 1, price: None, Error: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30068', Metadata(tracking_id='9e8ea6eceb8536f23a933406b40e0e8a', ratelimit_limit='300, 300;w=60', ratelimit_remaining=298, ratelimit_reset=52, message='Only limit order is allowed'))


None


In [18]:
with app.app_context():
    buy_l_res = bot.trade_strategy.buy_limit(137.6, 1)
    print(buy_l_res)

PostOrderResponse(order_id='REXTYB1DULR1P0NY8DH3', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_NEW: 4>, lots_requested=1, lots_executed=0, initial_order_price=MoneyValue(currency='rub', units=137, nano=600000000), executed_order_price=MoneyValue(currency='', units=0, nano=0), total_order_amount=MoneyValue(currency='rub', units=137, nano=600000000), initial_commission=MoneyValue(currency='', units=0, nano=0), executed_commission=MoneyValue(currency='', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG00F9XX7H4', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=137, nano=600000000), order_type=<OrderType.ORDER_TYPE_LIMIT: 1>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='c7485564-ed92-45fd-a724-1214aa202904', order_request_id='2024-11-17 12:31:29.838895+00:00', response_metadata=ResponseMetadata(tracking_id='58c12d40d1032f8b15ed98ae973b3

In [21]:
with app.app_context():
    buy_res = bot.trade_strategy.buy_bestprice(1)
    print(buy_res)

2024-11-17 15:35:55,765 - INFO - BUY BESTPRICE executed, 1 x -137.9 | s2 (x1=2) | p 0.3 rub


PostOrderResponse(order_id='REX4J6MVQ24AW79IYDKW', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=138, nano=350000000), executed_order_price=MoneyValue(currency='rub', units=137, nano=900000000), total_order_amount=MoneyValue(currency='rub', units=137, nano=900000000), initial_commission=MoneyValue(currency='rub', units=0, nano=0), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG00F9XX7H4', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=138, nano=350000000), order_type=<OrderType.ORDER_TYPE_BESTPRICE: 3>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='c7485564-ed92-45fd-a724-1214aa202904', order_request_id='2024-11-17 12:35:54.430818+00:00', response_metadata=ResponseMetadata(tracking_id='43e30

In [20]:
with app.app_context():
    buy_res = bot.trade_strategy.sell_bestprice(1)
    print(buy_res)

2024-11-17 15:35:44,429 - INFO - SELL BESTPRICE executed, 3 x 137.8 | s1 (x1=1) | p 0.6 rub


PostOrderResponse(order_id='REXC7IMBPO2Y6TKZIBCT', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=3, lots_executed=3, initial_order_price=MoneyValue(currency='rub', units=412, nano=50000000), executed_order_price=MoneyValue(currency='rub', units=137, nano=800000000), total_order_amount=MoneyValue(currency='rub', units=413, nano=400000000), initial_commission=MoneyValue(currency='rub', units=0, nano=0), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG00F9XX7H4', direction=<OrderDirection.ORDER_DIRECTION_SELL: 2>, initial_security_price=MoneyValue(currency='rub', units=137, nano=350000000), order_type=<OrderType.ORDER_TYPE_BESTPRICE: 3>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='c7485564-ed92-45fd-a724-1214aa202904', order_request_id='2024-11-17 12:35:43.150139+00:00', response_metadata=ResponseMetadata(tracking_id='0ae83